In [1]:
import os 
import sys 
import argparse
import gzip 
import json
import tqdm 
import numpy as np 
import open3d as o3d 
import plotly 
import pandas as pd 
import quaternion as qt 

import habitat_sim
from habitat.utils.visualizations import maps
from habitat.utils.visualizations.utils import images_to_video

# sys.path.append("/home/junting/project_cvl/SceneGraphNav")
%cd "/home/junting/project_cvl/SceneGraphNav"
from dataset.habitat.utils import read_house_file
from dataset.habitat.simulator import init_sim


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
/home/junting/project_cvl/SceneGraphNav


In [2]:
scene_error = ['HxpKQynjfin', '29hnd4uzFmX']
multi_layer_scene = ['XcA2TqTSSAj']
scene_empty_map = ['fzynW3qQPVF']
good_scene = "17DRP5sb8fy"

def set_path(args):
    
    args.scene_ply_path = os.path.join(args.scan_dir, args.scene_name, f'{args.scene_name}_semantic.ply')
    args.scene_glb_path = os.path.join(args.scan_dir, args.scene_name, f'{args.scene_name}.glb')
    args.scene_navmesh_path = os.path.join(args.scan_dir, args.scene_name, f'{args.scene_name}.navmesh')
    args.pclseg_path = os.path.join(args.scan_dir, args.scene_name, f'{args.scene_name}_pclseg.txt')
    args.pcl_normals_path = os.path.join(args.scan_dir, args.scene_name, f'{args.scene_name}_normals.npy')
    args.house_file_path = os.path.join(args.scan_dir, args.scene_name, f'{args.scene_name}.house')
    
    if os.path.exists(
        os.path.join(args.task_dir, 'train', 'content', f'{args.scene_name}.json.gz')
    ): # train split 
        args.episode_path = os.path.join(args.task_dir, 'train', 'content', f'{args.scene_name}.json.gz') 
        # print(f"scene {args.scene_name} is in train split")
    elif os.path.exists(
        os.path.join(args.task_dir, 'val', 'content', f'{args.scene_name}.json.gz')
    ): # val split 
        args.episode_path = os.path.join(args.task_dir, 'val', 'content', f'{args.scene_name}.json.gz') 
        # print(f"scene {args.scene_name} is in val split")
    else:
        print(f"No episode file found for scene {args.scene_name}")
        args.episode_path = None


def parse_args(input=""):

    parser = argparse.ArgumentParser()
    parser.add_argument("--scan_dir", type=str, default="/media/junting/SSD_data/habitat_data/scene_datasets/mp3d/v1/scans")
    parser.add_argument("--task_dir", type=str, default="/media/junting/SSD_data/habitat_data/datasets/objectnav/mp3d/v1")
    parser.add_argument("--scene_name", type=str, default="1LXtFkjw3qL") # 17DRP5sb8fy # XcA2TqTSSAj
    parser.add_argument("--dataset", type=str, default="matterport")
    # parser.add_argument("--split")

    parser.add_argument("--visualize", type=int, default=1)
    parser.add_argument("--vis_mode", type=str, default="segment", choices=["rgb", "segment"])
    
    parser.add_argument("--rel_dist_thresh", type=float, default=2.0, help="threshold of max distance between two objects that could have a relationship")

    args = parser.parse_args(input)  
    set_path(args)

    return args 

args = parse_args()

### Analysis: start & end positions

In [3]:
# get train & split set 
train_splits = [ file.split(".")[0]
    for file in os.listdir("/media/junting/SSD_data/habitat_data/datasets/objectnav/mp3d/v1/train/content")
]
val_splits = [ file.split(".")[0]
    for file in os.listdir("/media/junting/SSD_data/habitat_data/datasets/objectnav/mp3d/v1/val/content")
]

In [4]:
# # check if there are multi-level scene in datesets: YES ! 
# layer_dict = {}
# mul_scene = []
# for scene in train_splits:
#     args.scene_name = scene
#     set_path(args)
#     house_file, _ = read_house_file(args.house_file_path)
#     layer_dict[scene] = int(house_file['H'][0]['num_levels'])
# for k, v in layer_dict.items():
#     if v > 1:
#         print(k, f'has {v} layers')
#         mul_scene.append(k)

multi-layer scenes in 
```
['1LXtFkjw3qL', '1pXnuDYAj8r', '29hnd4uzFmX', '5LpN3gDmAk7', '5q7pvUzZiYa', '759xd9YjKW5', '7y3sRwLe3Va', '82sE5b5pLXE', 'aayBHfsNo7d', 'b8cTxDM8gDG', 'cV4RVeZvu5T', 'D7G3Y4RVNrH', 'D7N2EKCX4Sj', 'E9uDoFAP3SH', 'e9zR4mvMWw7', 'EDJbREhghzL', 'i5noydFURQK', 'JF19kD82Mey', 'kEZ7cmS4wCh', 'p5wJjkQkbXX', 'Pm6F8kyY3z2', 'pRbA3pwrgk9', 'PX4nDJXEHrG', 'qoiz87JEwZ2', 'rPc6DW4iMge', 's8pcmisQ38h', 'S9hNv5qa7GM', 'sKLMLpTHeUy', 'ULsKaCPVFJR', 'uNb9QFRL6hY', 'V2XKFyX4ASd', 'VFuaQ6m2Qom', 'VLzqgDo317F', 'VVfe2KiqLaN', 'vyrNrziPKCB', 'XcA2TqTSSAj', 'ZMojNkEp431']
```

In [5]:
# annot_file = annot_files[0]
# args.scene_name = train_splits[0]
# print(args.scene_name)
# set_path(args)

with gzip.open(args.episode_path, 'r') as json_file:
    scene_task_data = json.loads(json_file.read().decode('utf-8'))
print(scene_task_data.keys())
# print(scene_task_data['goals_by_category']['HxpKQynjfin.glb_table'][0])
# HxpKQynjfin_glb_table = scene_task_data['goals_by_category']['HxpKQynjfin.glb_table'][0]
# print(HxpKQynjfin_glb_table.keys())
# print(HxpKQynjfin_glb_table['object_id'])
print(scene_task_data['episodes'][0])

dict_objectid2goal = {
    goal['object_id']: goal
    for cat in scene_task_data['goals_by_category'].keys()
    for goal in scene_task_data['goals_by_category'][cat]
}

dict_keys(['goals_by_category', 'episodes', 'category_to_task_category_id', 'category_to_mp3d_category_id'])
{'episode_id': '0', 'scene_id': 'mp3d/1LXtFkjw3qL/1LXtFkjw3qL.glb', 'start_position': [-5.73303, 0.08441, 6.67048], 'start_rotation': [0, 0.92732, 0, -0.37428], 'info': {'geodesic_distance': 24.10707, 'euclidean_distance': 4.77366, 'closest_goal_object_id': 6, 'navigation_bounds': [[-7.62605, -3.11559, -4.36096], [4.15602, 10.20153, 17.4473]], 'best_viewpoint_position': [-2.51318, 3.48441, 5.88465]}, 'goals': [], 'start_room': None, 'shortest_paths': [[3, 3, 3, 1, 3, 1, 2, 2, 2, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 3, 3, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1, 3, 3, 1, 3, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 3, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 2, None]], 'object_c

In [6]:
init_positions = []
init_heights = []
for episode in scene_task_data['episodes']:
    init_positions.append(episode['start_position'])
    init_heights.append(episode['start_position'][1])
   
# print 
# heights, counts = np.unique(init_heights, return_counts=True)
# sort_idx = np.argsort(-counts)
# counts = counts[sort_idx]
# heights = heights[sort_idx]
# print({heights[i]:counts[i]
#     for i in range(len(heights))
# })

# visualize 
import plotly.express as px
positions = np.array(init_positions)
df = pd.DataFrame({
    'x': positions[:,0],
    'y': positions[:,1],
    'z': positions[:,2],
})
fig = px.histogram(
    df, 
    x='y', # height is y-axis
    title='height distribution of episode init position',
    labels={'y':'height'}) 
fig.show()

In [7]:
# # initial habitat sim ONLY load navmesh to save time 
# import habitat_sim

# # parameter 
# move_filter_fn = "try_step"

# pathfinder = habitat_sim.PathFinder()
# pathfinder.load_nav_mesh(args.scene_navmesh_path)
# assert pathfinder.is_loaded
# pathfinder.seed(0)
# np.random.seed(seed=0)

# scene_graph = habitat_sim.SceneGraph()
# agent = habitat_sim.Agent(scene_graph.get_root_node().create_child())
# agent.controls.move_filter_fn = getattr(pathfinder, move_filter_fn)

# # agent.agent_config.action_space["turn_left"].actuation.amount = TURN_DEGREE
# # agent.agent_config.action_space["turn_right"].actuation.amount = TURN_DEGREE

# follower = habitat_sim.GreedyGeodesicFollower(
#     pathfinder,
#     agent,
#     forward_key="move_forward",
#     left_key="turn_left",
#     right_key="turn_right",
# )

In [8]:
# initialize the full habitat environment 
try:  # Got to make initialization idiot proof
    sim.close()
except NameError:
    pass
sim, action_names, sim_settings = init_sim(args.scene_glb_path)
pathfinder = sim.pathfinder

I0307 14:47:34.970547 1007524 ManagedFileBasedContainer.h:210] <Dataset>::convertFilenameToPassedExt : Filename : default changed to proposed scene_dataset_config.json filename : default.scene_dataset_config.json
I0307 14:47:34.970573 1007524 AttributesManagerBase.h:365] <Dataset>::createFromJsonOrDefaultInternal : Proposing JSON name : default.scene_dataset_config.json from original name : default | This file  does not exist.
I0307 14:47:34.970634 1007524 AssetAttributesManager.cpp:120] Asset attributes (capsule3DSolid : capsule3DSolid_hemiRings_4_cylRings_1_segments_12_halfLen_0.75_useTexCoords_false_useTangents_false) created and registered.
I0307 14:47:34.970662 1007524 AssetAttributesManager.cpp:120] Asset attributes (capsule3DWireframe : capsule3DWireframe_hemiRings_8_cylRings_1_segments_16_halfLen_1) created and registered.
I0307 14:47:34.970690 1007524 AssetAttributesManager.cpp:120] Asset attributes (coneSolid : coneSolid_segments_12_halfLen_1.25_rings_1_useTexCoords_false_use

Renderer: NVIDIA GeForce RTX 3080/PCIe/SSE2 by NVIDIA Corporation
OpenGL version: 4.6.0 NVIDIA 470.103.01
Using optional features:
    GL_ARB_vertex_array_object
    GL_ARB_ES2_compatibility
    GL_ARB_separate_shader_objects
    GL_ARB_robustness
    GL_ARB_texture_storage
    GL_ARB_invalidate_subdata
    GL_ARB_texture_storage_multisample
    GL_ARB_multi_bind
    GL_ARB_direct_state_access
    GL_ARB_get_texture_sub_image
    GL_ARB_texture_filter_anisotropic
    GL_KHR_debug
Using driver workarounds:
    no-forward-compatible-core-context
    no-layout-qualifiers-on-old-glsl
    nv-zero-context-profile-mask
    nv-implementation-color-read-format-dsa-broken
    nv-cubemap-inconsistent-compressed-image-size
    nv-cubemap-broken-full-compressed-image-query
    nv-compressed-block-size-in-bits
agent_state: position [0. 0. 0.] rotation quaternion(1, 0, 0, 0)
sensor_states: {'rgb': SixDOFPose(position=array([0. , 1.5, 0. ], dtype=float32), rotation=quaternion(1, 0, 0, 0)), 'depth': Si

oposing JSON name : /media/junting/SSD_data/habitat_data/scene_datasets/mp3d/v1/scans/1LXtFkjw3qL/1LXtFkjw3qL.stage_config.json from original name : /media/junting/SSD_data/habitat_data/scene_datasets/mp3d/v1/scans/1LXtFkjw3qL/1LXtFkjw3qL.glb | This file  does not exist.
I0307 14:47:35.135360 1007524 AbstractObjectAttributesManagerBase.h:183] File (/media/junting/SSD_data/habitat_data/scene_datasets/mp3d/v1/scans/1LXtFkjw3qL/1LXtFkjw3qL.glb) exists but is not a recognized config filename extension, so new default Stage Template attributes created and registered.
I0307 14:47:35.135385 1007524 SceneDatasetAttributes.cpp:45] ::addNewSceneInstanceToDataset : Dataset : 'default' : Stage Attributes '/media/junting/SSD_data/habitat_data/scene_datasets/mp3d/v1/scans/1LXtFkjw3qL/1LXtFkjw3qL.glb' specified in Scene Attributes exists in dataset library.
I0307 14:47:35.135387 1007524 SceneDatasetAttributes.cpp:79] ::addNewSceneInstanceToDataset : Dataset : 'default' : Lighting Layout Attributes 'n

In [14]:
# query initial position island_radius in habitat simulator 
pos_navmesh_rad = []
for init_pos in positions:
    pos_navmesh_rad.append(pathfinder.island_radius(init_pos))

In [15]:
# import plotly.express as px
df2 = pd.DataFrame({'rad': pos_navmesh_rad})
fig = px.histogram(
    df2, 
    x='rad', # height is y-axis
    title='navmesh radius distribution of episode init positions',
    labels={'rad':'island radius'}) 
fig.show()

In [23]:
# get start & end point
import shutil
OUTPUT_VIDEO = True
agent = sim.get_agent(0) # default agent 
actions = ['stay', 'move_forward', 'turn_left', 'turn_right']

# def draw_top_down_map(info, output_size):
#     return maps.colorize_draw_agent_and_fit_to_height(
#         info["top_down_map"], output_size
#     )
end_position = []


for ep_idx, episode in enumerate(tqdm.tqdm(scene_task_data['episodes'], desc="running episodes:")):
    
    if OUTPUT_VIDEO: # DEBUG mode 
        ep_idx = 0
        episode = scene_task_data['episodes'][ep_idx]
        dirname = os.path.join(
            "./output", "gt_episode_video", f"ep_{ep_idx:02d}")
        if os.path.exists(dirname):
            shutil.rmtree(dirname)
        os.makedirs(dirname)
        
    pos = episode['start_position']
    # print("initial pos:", pos)
    rot = episode['start_rotation']
    init_state = habitat_sim.AgentState(
        position=np.array(pos),
        rotation=np.quaternion(rot[3], rot[0], rot[1], rot[2])
    )
    agent.set_state(init_state)

    images = []
    for action_label in episode['shortest_paths'][0]:
        if action_label != None:
            action_label = int(action_label)
            if action_label < 4: # ignore look_up or look_down
            
                action = actions[action_label]
                observations = sim.step(action)
                im = observations["rgb"]
                # top_down_map = draw_top_down_map(info, im.shape[0])
                # output_im = np.concatenate((im, top_down_map), axis=1)
                if OUTPUT_VIDEO:
                    output_im = im
                    images.append(output_im)

    if OUTPUT_VIDEO:
        images_to_video(images, dirname, "trajectory")
        break 
    # print("Episode finished")
    end_state = agent.get_state()
    end_position.append(end_state.position)
    
        



running episodes::   0%|          | 0/49997 [00:02<?, ?it/s]


In [21]:
for i, end_pos in enumerate(end_position):
    diff = end_pos - positions[i]
    if diff[1] > 0.5:
        print(i, diff)

0 [ 3.24684636  3.43772359 -0.74852394]
2 [-3.78249745  2.48235094  5.60270545]
5 [0.28950658 2.87184129 7.71478837]
8 [ 3.19800104  3.41795487 -6.88376978]
10 [ 4.46281167  3.42857332 -4.11350082]
15 [-4.42555857  2.317401    5.76579968]
18 [-1.72179468  2.82652173  8.73292117]
19 [-2.36663768  2.84434209  4.93562338]
20 [-2.82926668  2.85303435  8.97187656]
21 [ 1.58760498  3.400001   -2.1205709 ]
23 [ 4.24881648  3.46419616 -7.74418087]
26 [1.92617436 2.8432332  5.48656782]
27 [ 2.63498529  3.46159453 -8.68631257]
29 [ 2.03328852  3.47221918 -3.55916878]
30 [ 2.499723    3.43282909 -3.17223277]
34 [2.73578699 3.00674487 2.64597939]
38 [ 4.33607842  3.400001   -7.92045961]
41 [ 1.60669212  3.45436473 -5.49647706]
44 [-2.65085577  2.83783565  8.45229577]
45 [2.1221238  3.31461442 1.45780656]
46 [-3.8715589   0.50793503  3.23269137]
48 [0.49145144 2.86587486 6.76202859]
49 [  3.11989382   0.60000105 -10.49787354]
52 [1.95912827 2.85804177 3.48939229]
53 [-0.22778213  2.84057579  8.5779

episode with height difference 
```
0 [ 3.24684636  3.43772359 -0.74852394]
2 [-3.78249745  2.48235094  5.60270545]
5 [0.28950658 2.87184129 7.71478837]
8 [ 3.19800104  3.41795487 -6.88376978]
10 [ 4.46281167  3.42857332 -4.11350082]
15 [-4.42555857  2.317401    5.76579968]
18 [-1.72179468  2.82652173  8.73292117]
19 [-2.36663768  2.84434209  4.93562338]
20 [-2.82926668  2.85303435  8.97187656]
21 [ 1.58760498  3.400001   -2.1205709 ]
23 [ 4.24881648  3.46419616 -7.74418087]
26 [1.92617436 2.8432332  5.48656782]
27 [ 2.63498529  3.46159453 -8.68631257]
29 [ 2.03328852  3.47221918 -3.55916878]
30 [ 2.499723    3.43282909 -3.17223277]
34 [2.73578699 3.00674487 2.64597939]
38 [ 4.33607842  3.400001   -7.92045961]
41 [ 1.60669212  3.45436473 -5.49647706]
44 [-2.65085577  2.83783565  8.45229577]
45 [2.1221238  3.31461442 1.45780656]
46 [-3.8715589   0.50793503  3.23269137]
48 [0.49145144 2.86587486 6.76202859]
49 [  3.11989382   0.60000105 -10.49787354]
52 [1.95912827 2.85804177 3.48939229]
53 [-0.22778213  2.84057579  8.57793602]
...
836 [-3.94440685  2.61345129  6.01025773]
837 [ 3.87654225  3.46275825 -1.73718549]
842 [0.5466994  2.84806261 7.49405694]
844 [ 3.83097216  3.43843145 -7.65505626]
```